# Part5

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
import requests, json, os, sys, time, re
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
import pandas as pd

Spark session is a unified entry point of a spark application from Spark 2.0. It provides a way to interact with various spark’s functionality with a lesser number of constructs. Instead of having a spark context, hive context, SQL context, now all of it is encapsulated in a Spark session.

https://medium.com/@achilleus/spark-session-10d0d66d1d24

In [3]:
######## Spark session is a unified entry point of a spark application  #############
spark = SparkSession \
    .builder \
    .appName('spark-rec') \
    .config("configuration_key", "configuration_value") \
    .enableHiveSupport() \
    .getOrCreate()

In [4]:
sc = spark.sparkContext

In [5]:
import os
import math
import datetime
import pyspark.sql.functions as sf
from pyspark.sql.functions import desc
from pyspark.sql.window import Window
from pyspark import SparkConf, SparkContext
import itertools
from math import sqrt
from operator import add
from os.path import join, isfile, dirname
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.sql.types import TimestampType
datasets_path=os.getcwd() + "/RE_data"
#ratings_file = os.path.join(datasets_path, 'data/ratings.csv')
ratings_raw_data = sc.textFile("df_col_use_app_raings_new.csv")
ratings_raw_data_header = ratings_raw_data.take(1)[0]
ratings_data = ratings_raw_data.filter(lambda line: line != ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[1]),int(tokens[2]),int(float(tokens[3])))).cache()
######   we should assume each column here, by position [1],[2],[3] and data charactors
rddTraining, rddValidating, rddTesting = ratings_data.randomSplit([6,2,2], seed=1001)
#####   Split Training data, Validation data and Testing data by portion of 60%, 20%, 20%

* Build the Spark ALS model

In [8]:
model = ALS.trainImplicit(rddTraining, 10, 10,alpha=0.01)
testset = sc.parallelize([(3, 4000), (3, 15700)])
model.predictAll(testset).collect()

[Rating(user=3, product=15700, rating=0.9810856805017862),
 Rating(user=3, product=4000, rating=0.9865452677015587)]

* Get the games' rating predictions for users

In [9]:
# Calculate all predictions
rddTesting_map = rddTesting.map(lambda r: ((r[0], r[1]))) 
predictions = model.predictAll(rddTesting_map).map(lambda r: ((r[0], r[1]), (r[2]))) 
predictions.take(5)    ####### Output 5 results
#model.predictAll(rddTesting_map).collect()


[((6, 304050), 0.002139947627648353),
 ((0, 377160), 0.007977275220483526),
 ((0, 863550), 0.001358585669876844),
 ((3, 863550), 0.002033511545274952),
 ((2, 239140), 0.009374279388243645)]

* We get the pair of Rates and our Predictions 

In [16]:
rates_and_preds = rddTesting.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions) 
rates_and_preds.take(5)

[((0, 30), (3.0, 0.0010476633442347838)),
 ((0, 550), (5.0, 0.0023775055058238292)),
 ((0, 1046930), (2.0, 0.001234996461892332)),
 ((0, 304930), (1.0, 0.008420771427189)),
 ((0, 319630), (5.0, 0.0029466629846523484))]

* Show the final top 10 recommendation for 8 users 0 - 7

In [11]:
Spark_rec_list = []
for i in range(8):
    Spark_rec_list.append(model.recommendProducts(i,10))
Spark_rec_df = pd.DataFrame(Spark_rec_list)

Spark_rec_df

,0,1,2,3,4,5,6,7,8,9
0,"(0, 10, 1.0025446037445078)","(0, 230410, 1.0025446037445078)","(0, 261550, 1.0024448637445733)","(0, 582010, 1.0024448637445733)","(0, 271590, 0.9989298636405441)","(0, 380, 0.9956213735508213)","(0, 578080, 0.9915200120365106)","(0, 239140, 0.9893747138128672)","(0, 238960, 0.9891562483470906)","(0, 49520, 0.9883464942170095)"
1,"(1, 236390, 1.0031726782021004)","(1, 204360, 1.0030602638164425)","(1, 255710, 1.0029456762465654)","(1, 304050, 1.0027098186652512)","(1, 1046930, 0.9948421181430341)","(1, 218620, 0.9947677688526182)","(1, 359550, 0.9941006187916457)","(1, 578080, 0.9887139625177036)","(1, 431960, 0.9843851740179559)","(1, 730, 0.98412921822175)"
2,"(2, 224260, 0.998428577512311)","(2, 271590, 0.9927297963946669)","(2, 1085660, 0.9921189471140497)","(2, 105600, 0.9912045180565499)","(2, 377160, 0.9897751138470965)","(2, 292030, 0.9869064284761138)","(2, 431960, 0.9868604580202677)","(2, 578080, 0.9864966271865028)","(2, 72850, 0.9856518764337774)","(2, 730, 0.9850583373736064)"
3,"(3, 219740, 1.0126490176535217)","(3, 278360, 1.0121814595297949)","(3, 130, 1.0121814595297949)","(3, 291480, 1.0120195786921489)","(3, 242760, 1.000737682118511)","(3, 96000, 0.996886636027428)","(3, 105600, 0.9944765413254808)","(3, 550, 0.9915030301419845)","(3, 322330, 0.9881789091968294)","(3, 4000, 0.9865452677015587)"
4,"(4, 381210, 1.012527750737992)","(4, 555570, 1.012527750737992)","(4, 320, 1.012527750737992)","(4, 370910, 1.012527750737992)","(4, 240, 1.012527750737992)","(4, 227300, 1.012527750737992)","(4, 863550, 1.0123275294827123)","(4, 420, 1.0121235767586951)","(4, 221380, 1.0121235767586951)","(4, 80, 1.0121235767586951)"
5,"(5, 220, 1.0037649816357825)","(5, 252950, 0.9965285736117486)","(5, 10180, 0.9965253814433124)","(5, 208090, 0.9956514269715111)","(5, 96000, 0.9954949740502363)","(5, 377160, 0.9950209419983913)","(5, 363970, 0.9896568909369732)","(5, 550, 0.9879841277759822)","(5, 15700, 0.9872608812503634)","(5, 219640, 0.987251665808663)"
6,"(6, 291550, 1.0016006773598747)","(6, 700330, 1.0014967904837375)","(6, 813820, 1.0014967904837375)","(6, 444090, 1.0012829574247817)","(6, 438100, 1.001172864262725)","(6, 252950, 0.9961103222857031)","(6, 1085660, 0.9956840877323796)","(6, 10180, 0.9953394543087688)","(6, 218620, 0.9930973027280252)","(6, 359550, 0.9926886309179952)"
7,"(7, 413150, 1.003157626205771)","(7, 44350, 0.9939826195056498)","(7, 30, 0.9939816074497254)","(7, 1046930, 0.992744321297494)","(7, 219640, 0.9913024088871345)","(7, 377160, 0.9899291997845885)","(7, 252490, 0.988954082750516)","(7, 500, 0.9882847541825474)","(7, 8930, 0.9871743504914039)","(7, 105600, 0.9858374942897781)"


* Mesuare our recommendation results by applying RMSE

In [12]:
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()) 
print ('For testing data the RMSE is %s' % (error))


For testing data the RMSE is 3.606342878485228
